In [1]:
#import packages
import numpy as np
from random import random, seed
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [2]:
#define Franke function
def FrankeFunction(x,y): #still got one division in here
    term1 = 0.75*np.exp(-(0.25*(9*x-2)*(9*x-2)) - 0.25*((9*y-2)*(9*y-2)))
    term2 = 0.75*np.exp(-((9*x+1)*(9*x+1))/49.0 - 0.1*(9*y+1))
    term3 = 0.5*np.exp(-(9*x-7)*(9*x-7)*0.25 - 0.25*((9*y-3)*(9*y-3)))
    term4 = -0.2*np.exp(-(9*x-4)*(9*x-4) - (9*y-7)*(9*y-7))
    return term1 + term2 + term3 + term4

In [31]:
# create model matrix
# Form is: x0y0, x1y0, x2y0, x0y1, x1y1, x0y2
def Model(x,y,P): # P is polynomial degree
    m = len(x)*len(y) # number of equations
    t = sum(range(P+2)) # number of terms in polynomial
    X = np.zeros((m,t)) # Model matrix
    a = np.matrix.flatten(x)
    b = np.matrix.flatten(y)
    c = 0 #counter
    for i in range(P+1):
        for j in range(P+1-i):
            X[:,c] = a**j*b**i
            c +=1
    return X

In [4]:
#define R2 function
def R2(y_data, y_model):
    return 1 - np.sum((y_data - y_model) ** 2) / np.sum((y_data - np.mean(y_model)) ** 2)

In [5]:
#define MSE function
def MSE(y_data,y_model):
    n = np.size(y_model)
    return np.sum((y_data-y_model)**2)/n

In [6]:
#define variance function
def Var(y_data, y_model,P,X):
    N = len(y_data)
    covar = np.linalg.inv(X.T.dot(X))#sigma2
    vari = np.diagonal(covar)
    return vari

In [46]:
#Create random variables/predictors
np.random.seed(1234)
N = 200 #points along x and y axes

x = np.random.uniform(0,1,N)
y = np.random.uniform(0,1,N)

# x = np.arange(1,3,1)
# y = np.arange(3,5,1)

x, y = np.meshgrid(x,y,sparse=False)

#create datapoints/results
z = FrankeFunction(x, y)

# Create noise
Noise = 0.0*np.random.randn(N,N) #ask how noise should be added. NxN?

#add noise
z_n = z+Noise

#flatten for use in functions
z_n = np.matrix.flatten(z_n)
z = np.matrix.flatten(z)

In [62]:
MPD = 5 #maximal polynomial degree

X = Model(x,y,MPD)
#test to show it works below
# print(X)
# print(np.matrix.flatten(z))
# print(format(FrankeFunction(1,3),"1.8e"),format(FrankeFunction(2,3),"1.8e"),format(FrankeFunction(1,4),"1.8e"),format(FrankeFunction(2,4),"1.8e"))

In [63]:
# find parameters
beta = np.linalg.inv(X.T.dot(X)).dot(X.T).dot(z_n)

# make prediction
ztilde = X @ beta

In [64]:
######  Makes model matrix, looks different than mine #####
# form: x0y0, x1y0, x0y1, x2y0, x1y1, x0y2
# poly2 = PolynomialFeatures(degree=MPD)
# a = x[:,np.newaxis]
# b = y[:,np.newaxis]
# cc = np.c_[a,b]
# Xskl = poly2.fit_transform(cc) 
##########################################################
# THIS CELL IS CURRENTLY NOT IN USE #

In [65]:
# find parameters
clf = LinearRegression(fit_intercept=False) #False to not center data, i.e. intercept is not 0
clf.fit(X,z_n) 

#make prediction
zpredict = clf.predict(X)

In [66]:
# The mean squared error              
print("Mean squared error (self): %.5f" % MSE(z_n, ztilde))
print("Mean squared error (skl): %.5f" % mean_squared_error(z_n, zpredict))

# Explained variance score: 1 is perfect prediction     
print('R2 score (self): %.5f' %R2(z_n, ztilde))
print('R2 score (skl): %.5f' % r2_score(z_n, zpredict))

Mean squared error (self): 0.00179
Mean squared error (skl): 0.00179
R2 score (self): 0.97531
R2 score (skl): 0.97531


In [67]:
print(Var(z_n, ztilde, MPD,X))
print(Var(z_n,zpredict,MPD,X))

[7.55227583e-03 9.61238982e-01 2.15525118e+01 1.05272993e+02
 1.12662593e+02 1.71913430e+01 9.57850488e-01 1.25105603e+01
 5.57881794e+01 6.46391594e+01 1.28782868e+01 2.19658399e+01
 5.62624923e+01 5.42690697e+01 1.13243930e+01 1.06146184e+02
 6.43402927e+01 1.19071661e+01 1.10817552e+02 1.26556959e+01
 1.64554588e+01]
[7.55227583e-03 9.61238982e-01 2.15525118e+01 1.05272993e+02
 1.12662593e+02 1.71913430e+01 9.57850488e-01 1.25105603e+01
 5.57881794e+01 6.46391594e+01 1.28782868e+01 2.19658399e+01
 5.62624923e+01 5.42690697e+01 1.13243930e+01 1.06146184e+02
 6.43402927e+01 1.19071661e+01 1.10817552e+02 1.26556959e+01
 1.64554588e+01]


In [39]:
print(beta)
print(clf.coef_)

[ 0.99755741 -0.78936808 -1.10556944  0.78953868  1.28269674  2.14195012
  0.22522286 -6.31532846 -1.48818415  4.42296496]
[ 0.99755741 -0.78936808 -1.10556944  0.78953868  1.28269674  2.14195012
  0.22522286 -6.31532846 -1.48818415  4.42296496]


In [23]:
std = 1.96*np.sqrt(Var(z_n, ztilde, MPD,X))
print(std)

[0.10785097 0.29293747 0.25695537 0.31461105 0.23973657 0.27273455]


In [24]:
# 95% CI: mu +-1.96*sigma

In [25]:
np.sqrt(0.271633)

0.5211842284643694